In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer,BertModel
from sklearn.metrics import accuracy_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [32]:
tokenizer = BertTokenizer.from_pretrained('./bert-tokenizer')

In [6]:
class BertSentimentModel(nn.Module):
  def __init__(self,model_name='bert-base-uncased',num_labels=3):
    super(BertSentimentModel,self).__init__()
    self.bert = BertModel.from_pretrained(model_name)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(self.bert.config.hidden_size,num_labels)

  def forward(self,input_ids,attention_mask):
    outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask)
    return self.fc(self.dropout(outputs.pooler_output))

In [7]:
model = torch.load('saved_model/bert_sentiment_model.pth',map_location=torch.device('cpu'),weights_only=False)

In [ ]:
# Evaluate model
class New:
  def evaluate_text(texts,true_labels):
    model.eval()
    correct,total = 0,0
    label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

    with torch.no_grad():
      inputs = tokenizer(texts,padding='max_length',max_length=128,truncation=True,return_tensors='pt')
      input_ids,attention_mask = inputs['input_ids'].to(device),inputs['attention_mask'].to(device)
      outputs = model(input_ids,attention_mask)

      if isinstance(outputs,torch.Tensor):
        logits = outputs
      else:
        logits = outputs.logits

      probs = nn.functional.softmax(logits,dim=-1)
      preds = torch.argmax(logits,dim=1)

      for i in range(len(texts)):
        pred_label = preds[i].item()
        confidence = probs[i][pred_label].item()
        true_label = label_map[true_labels[i]]

        print(f'Text: {texts[i]}\nPredicted Label: {label_map[pred_label]}, True Label: ({true_label}), Confidence: {confidence}')
      
        if label_map[pred_label] == true_label:
          correct += 1
        total += 1

      accuracy = correct/total
      print(f'Evaluation Accuracy: {accuracy:.4f}')

In [150]:
test_texts = [
    'I Love this product! It works perfactly',
    'This is the worst service i have ever received',
    'The movie was ok, nothing special but not bad at all either.'
]

In [151]:
test_labels = [2,0,1]

In [152]:
evaluate_text(test_texts,test_labels)

Text: I Love this product! It works perfactly
Predicted Label: Positive, True Label: (Positive), Confidence: 0.9894918203353882
Text: This is the worst service i have ever received
Predicted Label: Negative, True Label: (Negative), Confidence: 0.999582827091217
Text: The movie was ok, nothing special but not bad at all either.
Predicted Label: Neutral, True Label: (Neutral), Confidence: 0.830588161945343
Evaluation Accuracy: 1.0000
